In [1]:
import sys
import os
parent_dir = os.path.abspath('..')
sys.path.append(parent_dir)
import system.robot as robotSystem
import system.utils.dynamic_properties as part_properties
import numpy as np
import casadi as ca

In [2]:
alpha = robotSystem.RobotDynamics()
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
path_to_urdf = os.path.join(
    project_root,
    'usage',
    'urdf','reach_alpha_5',
    'alpha_5_robot.urdf'
)
alpha.from_file(path_to_urdf)

root = "base_link"
tip = "alpha_standard_jaws_base_link"

kinematic_dict, K, P, L, D, C, g, B, qdd, joint_torque, sys_id_coeff , F_payload = alpha.build_model(root, tip, floating_base=True)
c_parms, m_params, I_params, fv_coeff, fs_coeff, vec_g, r_com_body, m_p, q, q_dot, q_dotdot, tau, base_pose, world_pose = kinematic_dict['parameters']
n_joints = kinematic_dict['n_joints']
N = alpha.get_N
D_dot_2C = alpha.get_D_dot_2C
H = alpha.get_total_energy
H_dot = alpha.get_total_power
F_next = alpha.get_forward_simulation

@1=(cos(world_psi)*cos(world_thet)), @2=(cos(base_rpy_2)*cos(base_rpy_1)), @3=(((cos(world_psi)*sin(world_thet))*sin(world_phi))-(sin(world_psi)*cos(world_phi))), @4=(sin(base_rpy_2)*cos(base_rpy_1)), @5=((sin(world_psi)*sin(world_phi))+((cos(world_psi)*cos(world_phi))*sin(world_thet))), @6=sin(base_rpy_1), @7=(((@1*@2)+(@3*@4))-(@5*@6)), @8=cos(q_0), @9=(((cos(base_rpy_2)*sin(base_rpy_1))*sin(base_rpy_0))-(sin(base_rpy_2)*cos(base_rpy_0))), @10=((cos(base_rpy_2)*cos(base_rpy_0))+((sin(base_rpy_0)*sin(base_rpy_1))*sin(base_rpy_2))), @11=(cos(base_rpy_1)*sin(base_rpy_0)), @12=(((@1*@9)+(@3*@10))+(@5*@11)), @13=sin(q_0), @14=(sin(world_psi)*cos(world_thet)), @15=((cos(world_psi)*cos(world_phi))+((sin(world_phi)*sin(world_thet))*sin(world_psi))), @16=(((sin(world_thet)*sin(world_psi))*cos(world_phi))-(cos(world_psi)*sin(world_phi))), @17=(((@14*@2)+(@15*@4))-(@16*@6)), @18=(((@14*@9)+(@15*@10))+(@16*@11)), @19=(cos(world_thet)*sin(world_phi)), @20=sin(world_thet), @21=(cos(world_thet)*cos

In [3]:
# # c , cpp or matlab code generation for forward simulation
# F_next.generate("Mnext.c")
# os.system("gcc -fPIC -shared Mnext.c -o libMnext.so")

In [4]:
L.size(), K.size(), P.size(), D.size(), g.size(), B.size()

((1, 1), (1, 1), (1, 1), (4, 4), (4, 1), (4, 1))

In [5]:
D_f = ca.Function('inertia_matrix', [ca.vertcat(*c_parms), 
                               ca.vertcat(*m_params),
                               ca.vertcat(*I_params),
                               vec_g,
                               q,
                               q_dot,
                               base_pose,
                               world_pose ],[K, D , C, g, D_dot_2C, N, P])

In [6]:
joint_min = np.array([1.00, 0.01, 0.01, 0.01])
joint_max = np.array([5.50, 3.40, 3.40, 5.70])
base_T0 = [0.190, 0.000, -0.120, 3.142, 0.000, 0.000] #floating base mount
base_T1 = [0.0, 0.000, 0.0, 0.0, 0.000, 0.000] #fixed base mount
base_origin = [0.0, 1.0, 0, 0, 0, 0]
q_sample = np.array([5.23716914, 1.52230767, 0.08661294, 4.24328965])
c_sample = [5e-3, -1e-3, 16e-3,
    73.563e-3, -0.091e-3, -0.734e-3,
    17e-3, -26e-3, 2e-3,
    -0.030e-3, -3e-3, -98e-3]
m_sample = [0.194, 0.429, 0.115, 0.333]
I_sample = [1e-2, 1e-2, 1e-2, 0, 0, 0, 1e-2, 1e-2, 1e-2, 0, 0, 0, 1e-2, 1e-2, 1e-2, 0, 0, 0, 1e-2, 1e-2, 1e-2, 0, 0, 0]
g_vec_sample = [0, 0, -9.81]
fv_sample = [0.1, 0.02, 0.1, 0.05]
fs_sample = [0.1, 0.04, 0.3, 0.03]
qdot_sample = [0.21, 0.9, 0.13, 0.14]
qdotdot_sample = [0.0, 0.0, 0.0, 0.0]
r_com_body_sample = [0.0, 0.0, 0.0]
m_p_sample = 0.0
kinetic_energy, robot_inertia_matrix, coriolis_matrix, gravity_vector, D_dot_2C_matrix, N_matrix, potential_energy = D_f(c_sample,
                                                                                                        m_sample,
                                                                                                        I_sample,
                                                                                                        g_vec_sample,
    q_sample,
    qdot_sample,
    base_T0,
    base_origin
    )

kinetic_energy, robot_inertia_matrix, coriolis_matrix, potential_energy, gravity_vector

(DM(0.019291),
 DM(
 [[0.0616396, -0.000129443, 4.64784e-05, -0.00115267], 
  [-0.000129443, 0.0509071, -0.0249765, 0.000114339], 
  [4.64784e-05, -0.0249765, 0.0215426, -3.60267e-05], 
  [-0.00115267, 0.000114339, -3.60267e-05, 0.010003]]),
 DM(
 [[-0.00315522, -0.00036808, -0.000511898, 0.00380141], 
  [7.83225e-05, 2.46898e-06, -2.29406e-05, -0.000979342], 
  [0.000482987, 1.91134e-05, 1.35823e-06, 0.00102391], 
  [0.00378084, 0.000999568, -0.00103147, -1.18286e-18]]),
 DM(-7.63523),
 DM([2.88046, 0.68409, -0.402622, 4.44089e-16]))

In [7]:
# # *c_parms, *m_params, *I_params, fv_coeff, fs_coeff, vec_g, r_com_body, m_p, base_pose, world_pose
# p_sample = c_sample+ m_sample+ I_sample+fv_sample+ fs_sample+ g_vec_sample + r_com_body_sample + [m_p_sample] + base_T0 + base_origin
# p_sample

In [8]:
mrc_lump, I_lump  = sys_id_coeff['first_moments'],  sys_id_coeff['inertias_vec6']


mrc_I_i_fun = ca.Function('mrc_I_i', [ca.vertcat(*c_parms), 
                               ca.vertcat(*m_params),
                               ca.vertcat(*I_params),
                               q,
                               base_pose,
                               world_pose ], [ca.vertcat(*mrc_lump), ca.vertcat(*I_lump)])

            
m_id_vec = sys_id_coeff['masses_id_syms_vertcat']
m_rci_id_vec = sys_id_coeff['first_moments_id_vertcat']
I_id_vec = sys_id_coeff['inertias_id_vertcat']
fv_id_vec = sys_id_coeff['fv_id_vertcat']
fs_id_vec = sys_id_coeff['fs_id_vertcat']
 
reg_fun = ca.Function('reg', [m_id_vec, m_rci_id_vec, I_id_vec, vec_g, q,q_dot,base_pose,world_pose],[alpha.id_K, alpha.id_D, alpha.id_C, alpha.id_P, alpha.id_g])

mrc_lump_sample , I_lump_sample =  mrc_I_i_fun(c_sample, m_sample, I_sample ,q_sample, base_T0, base_origin)
id_kinetic_energy, id_robot_inertia_matrix, id_coriolis_matrix, id_potential_energy, id_gravity_vector = reg_fun(m_sample, mrc_lump_sample, I_lump_sample, g_vec_sample, q_sample, qdot_sample, base_T0, base_origin)

id_kinetic_energy, id_robot_inertia_matrix, id_coriolis_matrix, id_potential_energy, id_gravity_vector 

(DM(0.019291),
 DM(
 [[0.0616396, -0.000129443, 4.64784e-05, -0.00115267], 
  [-0.000129443, 0.0509071, -0.0249765, 0.000114339], 
  [4.64784e-05, -0.0249765, 0.0215426, -3.60267e-05], 
  [-0.00115267, 0.000114339, -3.60267e-05, 0.010003]]),
 DM(
 [[-0.00315522, -0.00036808, -0.000511898, 0.00380141], 
  [7.83225e-05, 2.46898e-06, -2.29406e-05, -0.000979342], 
  [0.000482987, 1.91134e-05, 1.35823e-06, 0.00102391], 
  [0.00378084, 0.000999568, -0.00103147, -1.59052e-18]]),
 DM(-7.63523),
 DM([2.88046, 0.68409, -0.402622, 0]))

In [9]:
tol = 1e-10
errs = {
    "K": float(ca.norm_inf(kinetic_energy - id_kinetic_energy)),
    "D": float(ca.norm_inf(robot_inertia_matrix - id_robot_inertia_matrix)),
    "C": float(ca.norm_inf(coriolis_matrix - id_coriolis_matrix)),
    "P": float(ca.norm_inf(potential_energy - id_potential_energy)),
    "g": float(ca.norm_inf(gravity_vector - id_gravity_vector)),
}
print(errs, {k: (v < tol) for k, v in errs.items()})


{'K': 3.469446951953614e-18, 'D': 8.294146619514109e-18, 'C': 5.800481622797449e-17, 'P': 0.0, 'g': 6.661338147750939e-16} {'K': True, 'D': True, 'C': True, 'P': True, 'g': True}


In [10]:
J_tip = kinematic_dict["geo_J"][-1]   # 6×n geometric Jacobian at the tool
SYS_ID_FUN = ca.Function('SYS_ID_TAU', [m_id_vec, m_rci_id_vec, I_id_vec, fv_id_vec,fs_id_vec, vec_g, r_com_body, m_p, q, q_dot, q_dotdot, base_pose,world_pose],[alpha.Y@alpha.id_theta + J_tip.T@F_payload])
SYS_ID_Y_FUN = ca.Function('SYS_ID_Y', [vec_g, r_com_body, m_p, q,q_dot, q_dotdot, base_pose,world_pose],[alpha.Y])

SYS_ID_FUN(m_sample, mrc_lump_sample, I_lump_sample, fv_sample, fs_sample, g_vec_sample, r_com_body_sample, m_p_sample, q_sample, qdot_sample, qdotdot_sample, base_T0, base_origin)

DM([3.00093, 0.741968, -0.0893603, 0.0385595])

In [11]:
Inverse_ID_FUN =ca.Function('Inverse_dynamics_tau', [ca.vertcat(*c_parms), ca.vertcat(*m_params), ca.vertcat(*I_params), fv_coeff, fs_coeff,
                                                     vec_g, r_com_body, m_p, q, q_dot, q_dotdot, base_pose, world_pose], [alpha.get_inverse_dynamics])
Inverse_ID_FUN(c_sample,m_sample,I_sample, fv_sample, fs_sample, g_vec_sample, r_com_body_sample, m_p_sample, q_sample, qdot_sample, qdotdot_sample, base_T0, base_origin)

DM([3.00093, 0.741968, -0.0893603, 0.0385595])

In [12]:
for i in range(10000):
    joint_min = np.array([1.00, 0.01, 0.01, 0.01])
    joint_max = np.array([5.50, 3.40, 3.40, 5.70])
    q_sample = np.random.uniform(joint_min, joint_max)

    kinetic_energy, robot_inertia_matrix, coriolis_matrix, gravity_vector, D_dot_2C_matrix, N_matrix, potential_energy = D_f(c_sample,
                                                                                                            m_sample,
                                                                                                            I_sample,
                                                                                                            g_vec_sample,
        q_sample,
        qdot_sample,
        base_T0,
        base_origin
        )

    checks = part_properties.is_spd_strict(robot_inertia_matrix) and part_properties.is_skew_symmetric(N_matrix) and np.allclose(D_dot_2C_matrix, N_matrix)
    if not checks:
        raise ValueError(f"check failed for q:{q_sample}")